# 🕵️ Agentic RAG using Data Agents

## 🧠 Concept

Data Agents are LLM-powered knowledge workers in LlamaIndex that can intelligently perform various tasks over your data, in both a "read" and "write" function. 

They are capable of:

- 🔍 Performing automated search and retrieval over different types of data - unstructured, semi-structured, and structured.

- 📞 Calling any external service API in a structured fashion, and processing the response + storing it for later.

In that sense, agents are a step beyond our query engines in that they can not only "read" from a static source of data, but can dynamically ingest and modify data from a variety of different tools.

## 🛠️ Building a Data Agent

Building a data agent requires the following core components:

1. 🌀 A reasoning loop
2. 🧰 Tool abstractions

A data agent is initialized with set of APIs, or Tools, to interact with; these APIs can be called by the agent to return information or modify state. Given an input task, the data agent uses a reasoning loop to decide which tools to use, in which sequence, and the parameters to call each tool.




In [ ]:
%%capture
!pip install llama-index==0.10.37 llama-index-embeddings-openai==0.1.9 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 llama-index-llms-openai==0.1.19

In [1]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv("")

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, setup_vector_store

In [2]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [3]:
# QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

QDRANT_URL=":memory:"

In [4]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [5]:
from llama_index.core.settings import Settings
from utils import setup_llm, setup_embed_model

setup_llm(
    provider="openai",
    api_key=OPENAI_API_KEY, 
    model="gpt-4o", 
    temperature=0.75, 
    system_prompt="""Use ONLY the provided context and generate a complete, coherent answer to the user's query. 
    Your response must be grounded in the provided context and relevant to the essence of the user's query.
    """
    )

setup_embed_model(
    provider="openai", 
    model="text-embedding-3-small",
    api_key=OPENAI_API_KEY
    )

In [6]:
import random
from llama_index.core.storage.docstore import SimpleDocumentStore
from utils import get_documents_from_docstore, group_documents_by_author, sample_documents

documents = get_documents_from_docstore("../data/words-of-the-senpais")

random.seed(42)

documents_by_author = group_documents_by_author(documents)

senpai_documents = sample_documents(documents_by_author, num_samples=10)

# Manually add some metadata to the nodes

In [7]:
known_for = {
    "Naval Ravikant": "Known for his insights on how to build wealth and achieve happiness through developing specific knowledge, embracing accountability, playing long-term games, and understanding the power of compound interest in all areas of life.",
    "Balaji Srinivasan": "Has insights on how to think independently, identify opportunities, and build a better future through the strategic application of technology and clear reasoning.",
    "Paul Graham": "Provides advice on the hacker mindset, arguing that hackers are really makers and creators - akin to painters - who can leverage their unique way of thinking to push boundaries, challenge the status quo, and shape the future through technology and entrepreneurship.",
    "Nassim Nicholas Taleb": "Argues for 'Skin in the Game', that is having a personal stake in the outcome is necessary for fairness as it aligns incentives and exposes individuals to both the potential rewards and risks of their decisions.",
    "Seneca": "Offers timeless advice on how to cultivate wisdom, build mental resilience, and live a life of purpose and contentment by focusing on what is essential, mastering one's emotions, and aligning oneself with nature.",
    "Bruce Lee": "Offers profound wisdom on self-improvement, personal growth, and martial arts philosophy, emphasizing the importance of adaptability, self-expression, and embracing one's own unique path in life, "
}

for document in senpai_documents:
    document.metadata['known_for'] = known_for.get(document.metadata['author']) 

In [8]:
smol_senpai_docstore = SimpleDocumentStore()
smol_senpai_docstore.add_documents(senpai_documents)

In [9]:
from utils import setup_vector_store

from llama_index.core import StorageContext
from llama_index.core.settings import Settings
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store.simple_index_store import SimpleIndexStore

COLLECTION_NAME = "agentic-rag"

vector_store = setup_vector_store(
    QDRANT_URL, 
    QDRANT_API_KEY, 
    COLLECTION_NAME, 
    enable_hybrid=True
    )

storage_context = StorageContext.from_defaults(
    docstore = smol_senpai_docstore,
    index_store=SimpleIndexStore(),
    vector_store = vector_store
    )

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

# Automatically add more metadata

In [10]:
from llama_index.core import PromptTemplate, VectorStoreIndex
from llama_index.core.schema import MetadataMode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.storage.docstore import SimpleDocumentStore

from llama_index.core.extractors import KeywordExtractor, QuestionsAnsweredExtractor

from utils import create_index, create_query_engine, ingest

from prompts import QUESTION_GEN_PROMPT

sentence_splitter = SentenceSplitter(chunk_size=256, chunk_overlap=16)

qa_extractor = QuestionsAnsweredExtractor(
    questions=2, 
    llm=Settings.llm, 
    metadata_mode=MetadataMode.EMBED,
    embed_model=Settings.embed_model,
    prompt_template=QUESTION_GEN_PROMPT
    )

keyword_extractor = KeywordExtractor(keywords=5, llm=Settings.llm)

In [11]:
transformations = [sentence_splitter, qa_extractor, keyword_extractor, Settings.embed_model]

nodes = ingest(
    documents=senpai_documents,
    transformations=transformations,
    vector_store=vector_store,   
)

index = create_index(
    from_where="vector_store",
    vector_store=vector_store,
    storage_context=storage_context,
    embed_model=Settings.embed_model,
    )

100%|██████████| 139/139 [00:19<00:00,  7.17it/s]


In [12]:
print(nodes[100].get_content(metadata_mode="all"))

[Excerpt from document]
page_number: 76
file_name: ../data/taoofseneca_vol2.pdf
title: Letters From a Stoic Volume 2
author: Seneca
known_for: Offers timeless advice on how to cultivate wisdom, build mental resilience, and live a life of purpose and contentment by focusing on what is essential, mastering one's emotions, and aligning oneself with nature.
questions_this_excerpt_can_answer: How can I achieve a state of happiness that is noble, steadfast, and calm?
excerpt_keywords: happiness, wisdom, resilience, emotions, nature
Excerpt:
-----
For its spirit is no less great and upright, its sagacity no less complete, its justice no less inflexible. It is, therefore, equally happy. For happiness has its abode in one place only, namely, in the mind itself, and is noble, steadfast, and calm; and this state cannot be attained without a knowledge of things divine and human. The other answer, which I promised to make to your objection, follows from this reasoning. The wise man is not distresse

To begin, we need to create a `VectorStoreInfo` object, which will contain all the relevant metadata for each component (in this case, title metadata). 

`VectorStoreInfo` is a component used in configuring vector stores within the LlamaIndex framework. It provides metadata about the content and structure of the data stored in the vector store. This metadata is helps indexing and querying processes, ensuring that the vector store is optimized for specific types of queries and data categories.

To use `VectorStoreInfo`, you need to define it with relevant metadata about your data. 

Here’s a step-by-step guide on how to configure and use `VectorStoreInfo`:

1. **Define Metadata Information**: Specify details about data categories and other relevant attributes.

In [13]:
from llama_index.core.vector_stores.types import VectorStoreInfo, MetadataInfo

vector_store_info = VectorStoreInfo(
    content_info="The thoughts and writings of philosophers, thinkers, and mentors who have shaped the world with their wisdom and insights.",
    metadata_info=[
        MetadataInfo(
            name="known_for",
            type="str",
            description="Useful for finding the right thinker's words to answer a question."
        ),
        MetadataInfo(
            name="questions_this_excerpt_can_answer",
            type="str",
            description="Use this when the user asks a question."
        ),
        MetadataInfo(
            name="excerpt_keywords",
            type="str",
            description="Use this when the user submits keywords.",
        ),
        MetadataInfo(
            name="author",
            type="str",
            description=("The name of the thinker who wrote the text. Useful when no text match is found, you can use internal knowledge to answer." )
            )
    ],
)

### Integration with Vector Index AutoRetriever

Next, we'll create our base Pydantic object to ensure compatibility with our 
application layer. This object verifies that the response from the OpenAI endpoint conforms to the specified schema.

After defining the `VectorStoreInfo`, you can use it to configure a `VectorIndexAutoRetriever`. This retriever automatically adjusts query settings based on the provided metadata.

In [14]:
from typing import List, Tuple, Any
from pydantic import BaseModel, Field

class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="A question the user wants to advice about")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[str] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names specified in filter_key_list)"
        )
    )


## Auto Retriever Functional Tool

This tool will leverage OpenAI's functional endpoint to select the correct metadata filter and query the filtered index - only looking at nodes with the desired metadata.


In [51]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.vector_stores.types import MetadataFilter, FilterOperator, MetadataFilters

top_k = 3

def auto_retrieve_fn(
    query: str, filter_key_list: List[str], filter_value_list: List[str]):
    """Auto retrieval function.

    Performs auto-retrieval from a vector database, and then applies a set of filters.

    """
    query = query or "Query"

    contains_filters = [
        MetadataFilter(key=k, value=v, operator=FilterOperator.CONTAINS)
        for k, v in zip(filter_key_list, filter_value_list)
        ]

    retriever = VectorIndexRetriever(
        index, 
        vector_store_query_mode="hybrid",
        alpha=0.65,
        filters=MetadataFilters(filters=contains_filters),
        top_k=top_k
        )

    query_engine = RetrieverQueryEngine.from_args(
        retriever,
        response_mode="compact",
        verbose=True
        )

    response = query_engine.query(query)
    return str(response)

In [52]:
from llama_index.core.tools import FunctionTool

description = f"""\
Use this tool to answer the usery query by retrieving relevant information from the vector database.
The vector database schema is given below, which you should use to find the right information to answer the user's query.:
{vector_store_info.json()}
"""

auto_retrieve_tool = FunctionTool.from_defaults(
    fn=auto_retrieve_fn,
    name="words-of-senpai-info",
    description=description,
    fn_schema=AutoRetrieveModel
)

In [53]:
from llama_index.agent.openai import OpenAIAgent

agent = OpenAIAgent.from_tools(
    tools=[auto_retrieve_tool],
    verbose=True,
)

## How does the agent work?

A user's query is processed through a query engine, transformed, and then filtered and retrieved from a vector database to provide a relevant response.

A simplified diagram: 

<img src="https://i.imgur.com/AICDPav.png" width="50%">

[Source: AI Makerspace](https://www.youtube.com/watch?v=wYZJq8CNmTw&t=3s)

1. **User Query**:
   - A user inputs a query.

2. **Query Engine**:
   - The query is first sent to the `QueryEngineRetriever` application, which is part of the Query Engine system.
   - The `QueryEngineRetriever` then interacts with the `VectorStoreData` application to process the query further.

3. **OpenAI API**:
   - The `VectorStoreData` communicates with two models from the OpenAI API:
     - **Language Model**: Understands and generate coherent responses based on the query.
     - **Embedding Model**: Converts the query and data into vector representations for easier processing and retrieval.

4. **Vector Store**:
   - The processed information is sent to a `Vector Database` within the Vector Store system.
   - The `Vector Database` stores the vectorized data and can filter it based on metadata through the `Metadata Filter` application.
   - This filtering process results in a `Filtered List of Nodes`, which represents the most relevant pieces of information related to the query.

5. **Result Compilation**:
   - The filtered information is sent back to the `VectorStoreData` application, which compiles the final response.
   - This compiled response is then sent back to the `QueryEngineRetriever` application.

6. **Response Display**:
   - Finally, the processed and relevant information is displayed to the user as the plot synopsis in the top right box.



In [54]:
agent.chat("Building wealth and achieving happiness through developing specific knowledge, embracing accountability, playing long-term games, and understanding the power of compound interest in all areas of life.")

Added user message to memory: Building wealth and achieving happiness through developing specific knowledge, embracing accountability, playing long-term games, and understanding the power of compound interest in all areas of life.
=== Calling Function ===
Calling function: words-of-senpai-info with args: {"query": "Building wealth through developing specific knowledge", "filter_key_list": ["questions_this_excerpt_can_answer"], "filter_value_list": ["How can I build wealth?"]}
Got output: To build wealth through developing specific knowledge, focus on acquiring skills that are unique and in demand but not easily teachable or replaceable. This specific knowledge often involves understanding complex systems, mastering advanced technologies, or developing niche expertise. As you gain this knowledge, combine it with increasing levels of leverage and accountability. Leverage can come from using tools such as software and capital, which allow you to amplify your efforts and achieve greater ou

AgentChatResponse(response="Here's a comprehensive approach to building wealth and achieving happiness through the principles of developing specific knowledge, embracing accountability, playing long-term games, and understanding the power of compound interest:\n\n### Building Wealth Through Developing Specific Knowledge\nTo build wealth, focus on acquiring skills that are unique, in demand, and not easily replaceable. This specific knowledge often involves understanding complex systems, mastering advanced technologies, or developing niche expertise. Combine this knowledge with increasing levels of leverage and accountability. Leverage can come from tools like software and capital, which amplify your efforts and lead to greater outcomes. Over time, continuously applying this specific knowledge and leveraging it effectively benefits from the compounding effects, leading to significant wealth creation.\n\n### Achieving Happiness Through Embracing Accountability\nHappiness is closely tied 

In [55]:
agent.chat("find text that mentions: specific knowledge, luck, success")

Added user message to memory: find text that mentions: specific knowledge, luck, success
=== Calling Function ===
Calling function: words-of-senpai-info with args: {"query":"specific knowledge, luck, success","filter_key_list":["excerpt_keywords"],"filter_value_list":["specific knowledge","luck","success"]}
Got output: Specific knowledge involves unique skills and abilities that are highly valuable and not easily replicable. When combined with leverage, such as technology or capital, and applied consistently over time, it increases the likelihood of achieving success. However, success also requires patience, as it often takes a long timescale to materialize. While luck can play a role, consistently applying specific knowledge with leverage increases your chances of success, even if it doesn't happen immediately.



AgentChatResponse(response='Here\'s an insightful excerpt that mentions specific knowledge, luck, and success:\n\n"Specific knowledge involves unique skills and abilities that are highly valuable and not easily replicable. When combined with leverage, such as technology or capital, and applied consistently over time, it increases the likelihood of achieving success. However, success also requires patience, as it often takes a long timescale to materialize. While luck can play a role, consistently applying specific knowledge with leverage increases your chances of success, even if it doesn\'t happen immediately."\n\nThis emphasizes the importance of developing unique skills, leveraging resources, and being patient to achieve long-term success, while acknowledging that luck can also influence outcomes.', sources=[ToolOutput(content="Specific knowledge involves unique skills and abilities that are highly valuable and not easily replicable. When combined with leverage, such as technology o

In [56]:
agent.chat("What would nassim taleb say about accountability and risk")

Added user message to memory: What would nassim taleb say about accountability and risk
=== Calling Function ===
Calling function: words-of-senpai-info with args: {"query":"accountability and risk","filter_key_list":["author"],"filter_value_list":["Nassim Taleb"]}
Got output: Accountability and risk are deeply intertwined concepts, especially when considering starting a company. Embracing accountability means being willing to take on the responsibility for the outcomes of your actions and decisions. It involves understanding that while there is inherent risk in starting a business—such as the time and capital you invest—modern society often mitigates the downside risk. For instance, even in the worst-case scenario of personal bankruptcy, the system can provide a fresh start. Furthermore, environments like Silicon Valley are generally forgiving of honest failures, provided you demonstrate high integrity and effort. This perspective encourages individuals to take on more accountability, 

AgentChatResponse(response='Nassim Taleb\'s perspective on accountability and risk highlights their deep interconnection, especially in contexts like starting a company. Here’s a key takeaway:\n\n"Embracing accountability means being willing to take on the responsibility for the outcomes of your actions and decisions. It involves understanding that while there is inherent risk in starting a business—such as the time and capital you invest—modern society often mitigates the downside risk. For instance, even in the worst-case scenario of personal bankruptcy, the system can provide a fresh start. Furthermore, environments like Silicon Valley are generally forgiving of honest failures, provided you demonstrate high integrity and effort. This perspective encourages individuals to take on more accountability, as the fear of failure is often less daunting than perceived."\n\nTaleb’s view emphasizes that taking risks is an integral part of being accountable, but societal safety nets and the cu

In [57]:
agent.chat("What would Bruce Lee say about adaptability and self-expression")

Added user message to memory: What would Bruce Lee say about adaptability and self-expression
=== Calling Function ===
Calling function: words-of-senpai-info with args: {"query":"adaptability and self-expression","filter_key_list":["author"],"filter_value_list":["Bruce Lee"]}
Got output: Adaptability and self-expression are essential components of personal growth and fulfillment. Adaptability allows you to remain flexible and responsive to changing circumstances, enabling a more fluid and effective approach to life's challenges. Self-expression, on the other hand, involves being true to your own thoughts, feelings, and identity, which fosters authenticity and creativity. Together, they help you navigate life's complexities while staying true to your unique path, ensuring a balanced and holistic approach to personal development.



AgentChatResponse(response='Bruce Lee emphasized the importance of adaptability and self-expression as key elements of personal growth and fulfillment. Here’s a synthesis of his teachings on these topics:\n\n"Adaptability allows you to remain flexible and responsive to changing circumstances, enabling a more fluid and effective approach to life\'s challenges. Self-expression, on the other hand, involves being true to your own thoughts, feelings, and identity, which fosters authenticity and creativity. Together, they help you navigate life\'s complexities while staying true to your unique path, ensuring a balanced and holistic approach to personal development."\n\nIn essence, Bruce Lee believed that being adaptable and expressing your true self are crucial for navigating life\'s uncertainties and achieving personal fulfillment.', sources=[ToolOutput(content="Adaptability and self-expression are essential components of personal growth and fulfillment. Adaptability allows you to remain fl

In [58]:
agent.chat("What kind of questions should I ask Balaji Srinivasan and Naval Ravikant?")

Added user message to memory: What kind of questions should I ask Balaji Srinivasan and Naval Ravikant?
=== Calling Function ===
Calling function: words-of-senpai-info with args: {"query": "What kind of questions should I ask Balaji Srinivasan?", "filter_key_list": ["author"], "filter_value_list": ["Balaji Srinivasan"]}
Got output: When engaging with Balaji Srinivasan, consider asking questions that delve into his areas of expertise, such as:

1. What strategic considerations should be made when evaluating emerging technology platforms like cryptocurrency and augmented reality (AR) glasses?
2. How can one effectively identify and leverage new opportunities in the tech industry?
3. What are the key factors to think about when assessing the potential of a new platform to solve significant pain points for users?
4. How do political dynamics influence the perception and acceptance of truth in society?
5. What strategies can individuals use to navigate political landscapes while maintaining

AgentChatResponse(response='### Questions for Balaji Srinivasan\n\n1. **Emerging Technologies**: What strategic considerations should be made when evaluating emerging technology platforms like cryptocurrency and augmented reality (AR) glasses?\n2. **Identifying Opportunities**: How can one effectively identify and leverage new opportunities in the tech industry?\n3. **Assessing Potential**: What are the key factors to think about when assessing the potential of a new platform to solve significant pain points for users?\n4. **Political Dynamics**: How do political dynamics influence the perception and acceptance of truth in society?\n5. **Navigating Political Landscapes**: What strategies can individuals use to navigate political landscapes while maintaining integrity and independence in their thinking?\n\n### Questions for Naval Ravikant\n\n1. **Developing Specific Knowledge**: How can I develop specific knowledge that will help me build wealth and achieve happiness?\n2. **Embracing Ac

### Lower-level API

<img src="https://docs.llamaindex.ai/en/stable/_static/agents/agent_step_execute.png" width="70%">


LlamaIndex "agents" are composed of `AgentRunner` objects that interact with `AgentWorkers`:

- **[`AgentRunner`](https://github.com/run-llama/llama_index/blob/157309347b85ececd32630906a356a64bc319282/llama-index-core/llama_index/core/agent/runner/base.py#L200)**: These are the orchestrators that manage state (including conversational memory), create and maintain tasks, execute steps within each task, and provide the high-level interface for user interaction.

- **[`AgentWorker`](https://github.com/run-llama/llama_index/blob/157309347b85ececd32630906a356a64bc319282/llama-index-legacy/llama_index/legacy/agent/types.py#L190)**: These control the step-wise execution of a task. Given an input step, an `AgentWorker` is responsible for generating the next step. They can be initialized with parameters and act on state passed down from the `Task` or `TaskStep` objects but do not inherently store state themselves. The outer `AgentRunner` calls an `AgentWorker` and collects/aggregates the results.


In [67]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [auto_retrieve_tool], 
    llm=Settings.llm, 
    verbose=True
)
agent_runner = AgentRunner(agent_worker, verbose=True)

In [68]:
agent_runner.chat("In what ways do Naval and Nassim think differently about the risk and accountability?")

> Running step 46ddad01-848c-4857-9e6f-84aaccffc39e. Step input: In what ways do Naval and Nassim think differently about the risk and accountability?
Added user message to memory: In what ways do Naval and Nassim think differently about the risk and accountability?
=== Calling Function ===
Calling function: words-of-senpai-info with args: {"query": "risk", "filter_key_list": ["author"], "filter_value_list": ["Naval Ravikant"]}
=== Function Output ===
Risk, in the context of evaluating decision-making processes, can significantly impact outcomes. For instance, if there is a possibility of ruin, traditional cost-benefit analyses become invalid because they cannot account for irreversible consequences. An example is playing Russian roulette; despite a high chance of immediate gain, the ultimate risk of fatality makes the expected return non-computable. In the realm of new tech businesses, regulatory risks are a critical factor, often outweighing technological barriers. Antiquated regulat

AgentChatResponse(response='assistant: Naval Ravikant and Nassim Nicholas Taleb offer nuanced perspectives on risk and accountability, each emphasizing different aspects of these concepts:\n\n### Naval Ravikant on Risk:\n- **Technological and Regulatory Risks**: Naval highlights the importance of regulatory risks, especially in the context of new tech businesses. He points out that outdated regulations can be significant barriers to innovation, as seen with companies like Uber and Airbnb. Despite technological readiness, the primary risks often come from navigating and overcoming these regulatory hurdles.\n- **Irreversible Consequences**: Naval also acknowledges that in high-stakes situations, such as playing Russian roulette, traditional cost-benefit analyses fail because they can\'t account for the irreversible negative outcomes.\n\n### Nassim Nicholas Taleb on Risk:\n- **Possibility of Ruin**: Taleb emphasizes the concept of "ruin," where the potential for catastrophic failure makes